In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.impute import SimpleImputer
import joblib
import os
import re
from scipy.sparse import save_npz, load_npz
import gc
print("Część 1: Import bibliotek do przygotowania danych - ZAKOŃCZONA.")

Część 1: Import bibliotek do przygotowania danych - ZAKOŃCZONA.


In [2]:
print("Krok 2: Konfiguracja środowiska")
ARTIFACTS_DIR = "Artifacts_Polska_v4"
DATA_DIR = os.path.join(ARTIFACTS_DIR, "preprocessed_data")
os.makedirs(DATA_DIR, exist_ok=True)

LOC_FILE = 'lokalizacja.csv'
DATA_FILE = 'saleflats_2024_2025.csv'
CLOSURES_FILE = 'LocationClosures.csv'
OUTPUT_FILE = 'Location_Polska_Predicted.csv'

MAX_TEXT_FEATURES = 10000 

Krok 2: Konfiguracja środowiska


In [3]:
print("--- Krok 3: Kompletne przygotowanie, podział i ZAPIS DANYCH ---")

# --- Sekcja 1: Wczytanie plików pomocniczych ---
print("Wczytywanie lokalizacja.csv i LocationClosures.csv...")
# Usunięto argument 'squeeze' i zamieniono go na operację .squeeze()
id_to_name = pd.read_csv(
    LOC_FILE, sep=',', header=None,
    names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'],
    usecols=['Id','Name'],
    index_col='Id',
    dtype={'Id': int, 'Name': str}
).squeeze('columns').to_dict()

# --- Sekcja 2: Wczytanie i przetworzenie głównego pliku danych ---
print(f"Wczytywanie i przetwarzanie {DATA_FILE}...")
try:
    first_row = pd.read_csv(DATA_FILE, sep=',', header=None, nrows=1, on_bad_lines='skip')
    last_col_index = first_row.shape[1] - 1
except Exception as e:
    last_col_index = 64
COLS_TO_EXTRACT = {3: 'Title', 4: 'Description', 5: 'Area', 6: 'Price', last_col_index: 'locationPath'}
df = pd.read_csv(DATA_FILE, sep=',', header=None, usecols=list(COLS_TO_EXTRACT.keys()), dtype=str, encoding='utf-8', on_bad_lines='skip')
df.rename(columns=COLS_TO_EXTRACT, inplace=True)
df.dropna(subset=['locationPath'], inplace=True)
df.drop_duplicates(subset=['Title', 'Description', 'locationPath'], keep='first', inplace=True)
df_original = df.copy()

# --- Sekcja 3: Parsowanie ścieżek i filtrowanie ---
print("Parsowanie ścieżek lokalizacji...")
path_cols = ['Wojewodztwo_ID', 'Powiat_ID', 'Gmina_ID', 'Miasto_ID', 'Dzielnica_ID', 'PodDzielnica_ID', 'Ulica_ID']
split_paths = df['locationPath'].str.split(',')
path_df = pd.DataFrame(split_paths.tolist(), index=df.index)
if path_df.shape[1] > 7: path_df = path_df.iloc[:, :7]
while path_df.shape[1] < 7: path_df[path_df.shape[1]] = None
path_df.columns = path_cols
path_df = path_df.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)
df = pd.concat([df.reset_index(drop=True), path_df.reset_index(drop=True)], axis=1)
df['target_district_id'] = np.where(df['PodDzielnica_ID'] != 0, df['PodDzielnica_ID'], df['Dzielnica_ID'])
df['target_city_id'] = df['Miasto_ID']
df['target_street_id'] = df['Ulica_ID']
df = df[df['target_city_id'] != 0].copy()
df.reset_index(drop=True, inplace=True)
print(f"Liczba ogłoszeń do dalszego przetwarzania: {len(df)}")

# --- Sekcja 4: Inżynieria Cech ---
print("Inżynieria cech (tekstowe i numeryczne)...")
num_features = ['Area', 'Price']
df[num_features] = df[num_features].apply(pd.to_numeric, errors='coerce')
imputer = SimpleImputer(strategy='median')
scaler = StandardScaler()
X_num = scaler.fit_transform(imputer.fit_transform(df[num_features]))
joblib.dump(scaler, os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))
joblib.dump(imputer, os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))
df['text_features'] = df['Title'].fillna('') + ' ' + df['Description'].fillna('')
vectorizer = TfidfVectorizer(max_features=MAX_TEXT_FEATURES, ngram_range=(1, 2))
X_text = vectorizer.fit_transform(df['text_features'])
joblib.dump(vectorizer, os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))

# --- Sekcja 5: Przygotowanie celów (target) ---
print("Przygotowanie zmiennych docelowych (y)...")
city_ids = sorted(df['target_city_id'].unique())
district_ids = sorted(df['target_district_id'].unique())
street_ids = sorted(df['target_street_id'].unique())
if 0 not in district_ids: district_ids.insert(0,0)
if 0 not in street_ids: street_ids.insert(0,0)
city_id_map = {id: i for i, id in enumerate(city_ids)}
district_id_map = {id: i for i, id in enumerate(district_ids)}
street_id_map = {id: i for i, id in enumerate(street_ids)}
X_city_feature = df['target_city_id'].map(city_id_map).values.astype(np.int32)
y_district = df['target_district_id'].map(district_id_map).values.astype(np.int32)
y_street = df['target_street_id'].map(street_id_map).values.astype(np.int32)
joblib.dump(city_id_map, os.path.join(ARTIFACTS_DIR, 'city_id_map.joblib'))
joblib.dump(district_id_map, os.path.join(ARTIFACTS_DIR, 'district_id_map.joblib'))
joblib.dump(street_id_map, os.path.join(ARTIFACTS_DIR, 'street_id_map.joblib'))
inv_district_id_map = {i: id for id, i in district_id_map.items()}
inv_street_id_map = {i: id for id, i in street_id_map.items()}
joblib.dump(inv_district_id_map, os.path.join(ARTIFACTS_DIR, 'inv_district_id_map.joblib'))
joblib.dump(inv_street_id_map, os.path.join(ARTIFACTS_DIR, 'inv_street_id_map.joblib'))


# --- Sekcja 6: Ręczny podział na zbiory treningowe i walidacyjne ---
print("Dzielenie danych na zbiory treningowe i walidacyjne...")
indices = np.arange(X_text.shape[0])
train_indices, val_indices = train_test_split(indices, test_size=0.2, random_state=42, shuffle=True)
X_text_train, X_text_val = X_text[train_indices], X_text[val_indices]
X_num_train, X_num_val = X_num[train_indices], X_num[val_indices]
X_city_train, X_city_val = X_city_feature[train_indices], X_city_feature[val_indices]
y_district_train, y_district_val = y_district[train_indices], y_district[val_indices]
y_street_train, y_street_val = y_street[train_indices], y_street[val_indices]
print(f"Podzielono na {X_text_train.shape[0]} próbek treningowych i {X_text_val.shape[0]} walidacyjnych.")

# --- Sekcja 7: ZAPIS DANYCH NA DYSK ---
print("Zapisywanie przetworzonych danych na dysk...")
save_npz(os.path.join(DATA_DIR, 'X_text_train.npz'), X_text_train)
save_npz(os.path.join(DATA_DIR, 'X_text_val.npz'), X_text_val)
np.save(os.path.join(DATA_DIR, 'X_num_train.npy'), X_num_train)
np.save(os.path.join(DATA_DIR, 'X_num_val.npy'), X_num_val)
np.save(os.path.join(DATA_DIR, 'X_city_train.npy'), X_city_train)
np.save(os.path.join(DATA_DIR, 'X_city_val.npy'), X_city_val)
np.save(os.path.join(DATA_DIR, 'y_district_train.npy'), y_district_train)
np.save(os.path.join(DATA_DIR, 'y_district_val.npy'), y_district_val)
np.save(os.path.join(DATA_DIR, 'y_street_train.npy'), y_street_train)
np.save(os.path.join(DATA_DIR, 'y_street_val.npy'), y_street_val)

# Zapisujemy df_original i df (przefiltrowany) do późniejszej predykcji
df_original.to_pickle(os.path.join(DATA_DIR, 'df_original.pkl'))
# POPRAWKA: Zapisujemy ramkę 'df', która jest naszą przefiltrowaną ramką danych.
df.to_pickle(os.path.join(DATA_DIR, 'df_filtered.pkl')) 
print("Wszystkie dane zostały przygotowane i zapisane.")
print("\n!!! WAŻNE !!!")
print("TERAZ PROSZĘ RĘCZNIE ZRESTARTOWAĆ KERNEL JUPYTERA.")
print("Użyj opcji 'Kernel -> Restart' w menu na górze.")
print("Po restarcie, uruchom komórki z CZĘŚCI 2.")

--- Krok 3: Kompletne przygotowanie, podział i ZAPIS DANYCH ---
Wczytywanie lokalizacja.csv i LocationClosures.csv...
Wczytywanie i przetwarzanie saleflats_2024_2025.csv...
Parsowanie ścieżek lokalizacji...
Liczba ogłoszeń do dalszego przetwarzania: 1092145
Inżynieria cech (tekstowe i numeryczne)...
Przygotowanie zmiennych docelowych (y)...
Dzielenie danych na zbiory treningowe i walidacyjne...
Podzielono na 873716 próbek treningowych i 218429 walidacyjnych.
Zapisywanie przetworzonych danych na dysk...
Wszystkie dane zostały przygotowane i zapisane.

!!! WAŻNE !!!
TERAZ PROSZĘ RĘCZNIE ZRESTARTOWAĆ KERNEL JUPYTERA.
Użyj opcji 'Kernel -> Restart' w menu na górze.
Po restarcie, uruchom komórki z CZĘŚCI 2.


### CZĘŚĆ 2 - zrestartuj KERNEL

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout, Concatenate, Embedding, Flatten
from scipy.sparse import load_npz
import joblib
import os
from tqdm.notebook import tqdm
import gc

print("--- Część 2: Wczytywanie przetworzonych danych ---")

# Konfiguracja ścieżek
ARTIFACTS_DIR = "Artifacts_Polska_v4"
DATA_DIR = os.path.join(ARTIFACTS_DIR, "preprocessed_data")

# Wczytywanie
X_text_train = load_npz(os.path.join(DATA_DIR, 'X_text_train.npz'))
X_text_val = load_npz(os.path.join(DATA_DIR, 'X_text_val.npz'))
X_num_train = np.load(os.path.join(DATA_DIR, 'X_num_train.npy'))
X_num_val = np.load(os.path.join(DATA_DIR, 'X_num_val.npy'))
X_city_train = np.load(os.path.join(DATA_DIR, 'X_city_train.npy'))
X_city_val = np.load(os.path.join(DATA_DIR, 'X_city_val.npy'))
y_district_train = np.load(os.path.join(DATA_DIR, 'y_district_train.npy'))
y_district_val = np.load(os.path.join(DATA_DIR, 'y_district_val.npy'))
y_street_train = np.load(os.path.join(DATA_DIR, 'y_street_train.npy'))
y_street_val = np.load(os.path.join(DATA_DIR, 'y_street_val.npy'))

# Wczytanie mapowań
city_id_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'city_id_map.joblib'))
district_id_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'district_id_map.joblib'))
street_id_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'street_id_map.joblib'))

print(f"Dane wczytane. Próbek treningowych: {X_text_train.shape[0]}")

--- Część 2: Wczytywanie przetworzonych danych ---
Dane wczytane. Próbek treningowych: 873716


In [2]:
print("\n--- Krok 5: Budowa i Kompilacja Modelu ---")

# Odczytanie wymiarów z wczytanych danych
MAX_TEXT_FEATURES = X_text_train.shape[1]
NUM_FEATURES = X_num_train.shape[1]
NUM_CITIES = len(city_id_map)
NUM_DISTRICTS = len(district_id_map)
NUM_STREETS = len(street_id_map)

input_text = Input(shape=(MAX_TEXT_FEATURES,), name='text_input', sparse=True)
input_num = Input(shape=(NUM_FEATURES,), name='num_input')
input_city = Input(shape=(1,), name='city_input')

x1 = Dense(128, activation='relu')(input_text)
x1 = Dropout(0.5)(x1)
x2 = Dense(64, activation='relu')(input_num)
x2 = Dense(32, activation='relu')(x2)
city_embedding_layer = Embedding(input_dim=NUM_CITIES, output_dim=50, name='city_embedding')(input_city)
x3 = Flatten()(city_embedding_layer)

combined = Concatenate()([x1, x2, x3])
z = Dense(256, activation='relu')(combined)
z = Dropout(0.5)(z)

output_district = Dense(NUM_DISTRICTS, activation='softmax', name='district_output')(z)
output_street = Dense(NUM_STREETS, activation='softmax', name='street_output')(z)

model = Model(inputs=[input_text, input_num, input_city], outputs=[output_district, output_street])
model.compile(optimizer='adam',
              loss={'district_output': 'sparse_categorical_crossentropy', 'street_output': 'sparse_categorical_crossentropy'},
              metrics={'district_output': 'accuracy', 'street_output': 'accuracy'})
model.summary()


--- Krok 5: Budowa i Kompilacja Modelu ---


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ text_input (InputLayer)       │ (None, 10000)             │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ num_input (InputLayer)        │ (None, 2)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_input (InputLayer)       │ (None, 1)                 │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense (Dense)                 │ (None, 128)               │       1,280,128 │ text_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_1 (Dense)               │ (None, 64)                │             192 │ num_input[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ city_embedding (Embedding)    │ (None, 1, 50)             │         415,900 │ city_input[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout (Dropout)             │ (None, 128)               │               0 │ dense[0][0]                │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_2 (Dense)               │ (None, 32)                │           2,080 │ dense_1[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ flatten (Flatten)             │ (None, 50)                │               0 │ city_embedding[0][0]       │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate (Concatenate)     │ (None, 210)               │               0 │ dropout[0][0],             │
│                               │                           │                 │ dense_2[0][0],             │
│                               │                           │                 │ flatten[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dense_3 (Dense)               │ (None, 256)               │          54,016 │ concatenate[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ dropout_1 (Dropout)           │ (None, 256)               │               0 │ dense_3[0][0]              │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ district_output (Dense)       │ (None, 2204)              │         566,428 │ dropout_1[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ street_output (Dense)         │ (None, 33888)             │       8,709,216 │ dropout_1[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 11,027,960 (42.07 MB)

 Trainable params: 11,027,960 (42.07 MB)

 Non-trainable params: 0 (0.00 B)

In [3]:
print("\n--- Krok 6: Trening Modelu ---")

X_train_list = [X_text_train, X_num_train, X_city_train]
y_train_dict = {'district_output': y_district_train, 'street_output': y_street_train}
X_val_list = [X_text_val, X_num_val, X_city_val]
y_val_dict = {'district_output': y_district_val, 'street_output': y_street_val}

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

history = model.fit(
    X_train_list, y_train_dict,
    validation_data=(X_val_list, y_val_dict),
    epochs=25,
    batch_size=128, # Używamy mniejszego batch_size dla bezpieczeństwa
    callbacks=[early_stopping]
)

model.save(os.path.join(ARTIFACTS_DIR, 'location_prediction_model.h5'))
print("Model został wytrenowany i zapisany.")


--- Krok 6: Trening Modelu ---
Epoch 1/25
6826/6826 ━━━━━━━━━━━━━━━━━━━━ 498s 73ms/step - district_output_accuracy: 0.4779 - district_output_loss: 2.8384 - loss: 6.7454 - street_output_accuracy: 0.6305 - street_output_loss: 3.9070 - val_district_output_accuracy: 0.5736 - val_district_output_loss: 1.6623 - val_loss: 4.2737 - val_street_output_accuracy: 0.6431 - val_street_output_loss: 2.6110
Epoch 2/25
6826/6826 ━━━━━━━━━━━━━━━━━━━━ 516s 76ms/step - district_output_accuracy: 0.5640 - district_output_loss: 1.6757 - loss: 4.2599 - street_output_accuracy: 0.6420 - street_output_loss: 2.5842 - val_district_output_accuracy: 0.6203 - val_district_output_loss: 1.3987 - val_loss: 3.5896 - val_street_output_accuracy: 0.6618 - val_street_output_loss: 2.1906
Epoch 3/25
6826/6826 ━━━━━━━━━━━━━━━━━━━━ 538s 79ms/step - district_output_accuracy: 0.5950 - district_output_loss: 1.4782 - loss: 3.6790 - street_output_accuracy: 0.6523 - street_output_loss: 2.2008 - val_district_output_accuracy: 0.6392 - v

Model został wytrenowany i zapisany.


In [4]:
print("\n--- Krok 7: Przygotowanie pełnego zbioru danych do predykcji ---")

# Konfiguracja
DATA_DIR = os.path.join(ARTIFACTS_DIR, "preprocessed_data")
CLOSURES_FILE = 'LocationClosures.csv'
LOC_FILE = 'lokalizacja.csv'

from sklearn.impute import SimpleImputer
df_filtered = pd.read_pickle(os.path.join(DATA_DIR, 'df_filtered.pkl'))
vectorizer = joblib.load(os.path.join(ARTIFACTS_DIR, 'vectorizer.joblib'))
scaler = joblib.load(os.path.join(ARTIFACTS_DIR, 'scaler.joblib'))
imputer = joblib.load(os.path.join(ARTIFACTS_DIR, 'imputer.joblib'))

# Przygotowanie pełnego zbioru cech
num_features = ['Area', 'Price']
df_filtered[num_features] = df_filtered[num_features].apply(pd.to_numeric, errors='coerce')
X_num_full = scaler.transform(imputer.transform(df_filtered[num_features]))
X_text_full = vectorizer.transform(df_filtered['text_features'])
X_city_full = df_filtered['target_city_id'].map(city_id_map).values.astype(np.int32)

# --- NOWA, WZBOGACONA LOGIKA HIERARCHII ---
print("Tworzenie WZBOGACONEJ hierarchii z dwóch źródeł...")

# 1. Wczytanie hierarchii z LocationClosures.csv (tak jak wcześniej)
df_lokalizacja = pd.read_csv(LOC_FILE, sep=',', header=None, names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'], usecols=['Id', 'AdditionalName'])
miasta_ids_set = set(df_lokalizacja[df_lokalizacja['AdditionalName'].str.contains('Miasto', na=False)]['Id'])
dzielnice_ids_set = set(df_lokalizacja[df_lokalizacja['AdditionalName'].str.contains('Dzielnica|Osiedle', na=False)]['Id'])
df_closures = pd.read_csv(CLOSURES_FILE)
rel_miasto_dzielnica_1 = df_closures[(df_closures['Depth'] == 1) & (df_closures['AncestorId'].isin(miasta_ids_set)) & (df_closures['DescendantId'].isin(dzielnice_ids_set))]
city_to_districts_1 = rel_miasto_dzielnica_1.groupby('AncestorId')['DescendantId'].apply(list).to_dict()
rel_dzielnica_ulica_1 = df_closures[(df_closures['Depth'] == 1) & (df_closures['AncestorId'].isin(dzielnice_ids_set))]
district_to_streets_1 = rel_dzielnica_ulica_1.groupby('AncestorId')['DescendantId'].apply(list).to_dict()

# 2. Wczytanie hierarchii dynamicznie z lokalizacja.csv (jak w modelu v3)
df_lok_full = pd.read_csv(LOC_FILE, sep=',', header=None, names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'], na_values=['\\N'])
df_lok_full.dropna(subset=['Id', 'ParentId'], inplace=True)
df_lok_full[['Id', 'ParentId']] = df_lok_full[['Id', 'ParentId']].astype(int)
city_to_districts_2 = df_lok_full[df_lok_full['Id'].isin(dzielnice_ids_set)].groupby('ParentId')['Id'].apply(list).to_dict()
district_to_streets_2 = df_lok_full[df_lok_full['AdditionalName'] == 'Ulica'].groupby('ParentId')['Id'].apply(list).to_dict()

# 3. Połączenie obu hierarchii w jedną kompletną
city_to_districts = city_to_districts_1.copy()
for city_id, districts in city_to_districts_2.items():
    if city_id in city_to_districts:
        city_to_districts[city_id] = list(set(city_to_districts[city_id] + districts))
    else:
        city_to_districts[city_id] = districts

district_to_streets = district_to_streets_1.copy()
for district_id, streets in district_to_streets_2.items():
    if district_id in district_to_streets:
        district_to_streets[district_id] = list(set(district_to_streets[district_id] + streets))
    else:
        district_to_streets[district_id] = streets

print("Słowniki hierarchii zostały połączone i wzbogacone.")
# --- KONIEC NOWEJ LOGIKI ---

# Wczytanie mapowań odwrotnych
inv_district_id_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'inv_district_id_map.joblib'))
inv_street_id_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'inv_street_id_map.joblib'))

print("Dane do predykcji gotowe.")


--- Krok 7: Przygotowanie pełnego zbioru danych do predykcji ---
Tworzenie WZBOGACONEJ hierarchii z dwóch źródeł...
Słowniki hierarchii zostały połączone i wzbogacone.
Dane do predykcji gotowe.


In [5]:
print("\n--- Krok 8: Predykcja na pełnym zbiorze z logiką hierarchiczną ---")

n_samples = X_text_full.shape[0]
chunk_size = 50000
predicted_district_ids = []
predicted_street_ids = []

for i in tqdm(range(0, n_samples, chunk_size)):
    chunk_list = [X_text_full[i:i + chunk_size], X_num_full[i:i + chunk_size], X_city_full[i:i + chunk_size]]
    pred_district_chunk, pred_street_chunk = model.predict(chunk_list, batch_size=512, verbose=0)
    
    for j in range(pred_district_chunk.shape[0]):
        city_id = df_filtered.iloc[i + j]['target_city_id']
        valid_districts = city_to_districts.get(city_id, [])
        pred_idx = np.argmax(pred_district_chunk[j])
        if valid_districts:
            mask = np.zeros_like(pred_district_chunk[j])
            valid_indices = [district_id_map.get(d_id) for d_id in valid_districts if district_id_map.get(d_id) is not None]
            if valid_indices:
                mask[valid_indices] = 1
                if np.sum(mask) > 0: pred_idx = np.argmax(pred_district_chunk[j] * mask)
        district_id = inv_district_id_map.get(pred_idx, 0)
        
        valid_streets = district_to_streets.get(district_id, [])
        pred_idx = np.argmax(pred_street_chunk[j])
        if valid_streets:
            mask = np.zeros_like(pred_street_chunk[j])
            valid_indices = [street_id_map.get(s_id) for s_id in valid_streets if street_id_map.get(s_id) is not None]
            if valid_indices:
                mask[valid_indices] = 1
                if np.sum(mask) > 0: pred_idx = np.argmax(pred_street_chunk[j] * mask)
        street_id = inv_street_id_map.get(pred_idx, 0)
        
        predicted_district_ids.append(district_id)
        predicted_street_ids.append(street_id)
    gc.collect()

df_filtered['predicted_district_id'] = predicted_district_ids
df_filtered['predicted_street_id'] = predicted_street_ids
print("Predykcja zakończona.")


--- Krok 8: Predykcja na pełnym zbiorze z logiką hierarchiczną ---


  0%|          | 0/22 [00:00<?, ?it/s]

Predykcja zakończona.


In [6]:
print("\n--- Krok 9: Generowanie i Zapis Wyników ---")

# Wczytanie potrzebnych obiektów
DATA_DIR = os.path.join(ARTIFACTS_DIR, "preprocessed_data")
df_original = pd.read_pickle(os.path.join(DATA_DIR, 'df_original.pkl'))
OUTPUT_FILE = 'Location_Polska_Predicted.csv'

# --- POPRAWKA: Używamy tego samego, pełnego słownika, co w komórce 10 ---
# Wczytujemy PEŁNY plik lokalizacji raz, aby mieć wszystkie potrzebne nazwy
df_lokalizacja_full = pd.read_csv('lokalizacja.csv', sep=',', header=None,
                                  names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'],
                                  na_values=['\\N'])
id_to_name = df_lokalizacja_full.set_index('Id')['Name'].to_dict()

# --- POPRAWKA: Funkcja tworząca string lokalizacji teraz używa tego samego, pełnego słownika ---
def create_loc_string(row):
    # Miasto bierzemy z oryginalnej, prawdziwej kolumny (target_city_id)
    # Dzielnicę i ulicę bierzemy z predykcji
    city_name = id_to_name.get(row['target_city_id'], "?")
    district_name = '?' if row['predicted_district_id'] == 0 else id_to_name.get(row['predicted_district_id'], "?")
    street_name = '?' if row['predicted_street_id'] == 0 else id_to_name.get(row['predicted_street_id'], "?")
    
    return f"{city_name} > {district_name} > {street_name}"

df_filtered['Predict_Loc'] = df_filtered.apply(create_loc_string, axis=1)

# Scalenie wyników z oryginalnym zbiorem
df_final = pd.merge(df_original, df_filtered[['Title', 'Description', 'locationPath', 'Predict_Loc']], on=['Title', 'Description', 'locationPath'], how='left')
df_final.drop_duplicates(subset=['Title', 'Description', 'locationPath'], inplace=True)
df_final['Predict_Loc'].fillna('Brak predykcji (brak miasta w danych wejściowych)', inplace=True)

# Zapis do pliku CSV
df_final.to_csv(OUTPUT_FILE, index=False, sep=';', encoding='utf-8-sig')

print(f"Zakończono! Wyniki zapisano do pliku: {OUTPUT_FILE}")


--- Krok 9: Generowanie i Zapis Wyników ---
Zakończono! Wyniki zapisano do pliku: Location_Polska_Predicted.csv


In [7]:
print("\nPrzykładowe 20 wierszy z wynikami:")

from IPython.display import display

# --- NOWA, POPRAWIONA FUNKCJA POMOCNICZA ---
# Wczytujemy PEŁNY plik lokalizacji raz, aby mieć wszystkie potrzebne nazwy
# POPRAWKA: Wczytujemy wszystkie kolumny i obsługujemy '\N' jako NaN
df_lokalizacja_full = pd.read_csv('lokalizacja.csv', sep=',', header=None, 
                                  names=['Id', 'ParentId', 'Name', 'AdditionalName', 'FullName'],
                                  na_values=['\\N']) # Traktuj '\N' jako brak danych

# Tworzymy słownik ID -> Nazwa na podstawie pełnego pliku
id_to_name_dict = df_lokalizacja_full.set_index('Id')['Name'].to_dict()

def translate_location_path(path_string, id_map):
    """Tłumaczy string '14,0,0,337,3257,0,0' na 'Małopolskie > ? > ? > Kraków > Krowodrza > ? > ?'"""
    if not isinstance(path_string, str):
        return "Błędny format ścieżki"
    
    names = []
    ids = path_string.split(',')
    
    for id_val_str in ids:
        # Sprawdzamy, czy wartość jest poprawną liczbą całkowitą i nie jest zerem
        try:
            id_val_int = int(id_val_str)
            if id_val_int == 0:
                names.append('?')
            else:
                # Pobieramy nazwę ze słownika, jeśli nie ma, dajemy '?'
                names.append(id_map.get(id_val_int, '?'))
        except (ValueError, TypeError):
            # Jeśli wartość nie jest liczbą (np. jest to pusty string lub coś innego)
            names.append('?')
            
    return ' > '.join(names)
# --- KONIEC FUNKCJI POMOCNICZEJ ---


predicted_df = df_final[df_final['Predict_Loc'].str.contains('>')]
if len(predicted_df) > 20:
    sample = predicted_df.sample(20, random_state=42)
else:
    sample = predicted_df.head(20)

# Dodajemy nową, przetłumaczoną kolumnę do wyświetlenia
sample['Original_Location_Name'] = sample['locationPath'].apply(lambda x: translate_location_path(x, id_to_name_dict))


# Zmieniamy kolejność kolumn dla lepszej czytelności
display_cols = ['Title', 'locationPath', 'Original_Location_Name', 'Predict_Loc']
sample_display = sample[display_cols]

# Stylizowanie
def highlight_col(s):
    colors = []
    for col_name in s.index:
        if col_name == 'Predict_Loc':
            colors.append('background-color: #aaffaa') # Zielony dla predykcji
        elif col_name == 'Original_Location_Name':
            colors.append('background-color: #ffff99') # Żółty dla oryginału
        else:
            colors.append('')
    return colors

styled_sample = sample_display.style.apply(highlight_col, axis=1)

# Ustawienie szerokości kolumn i wyrównania
styled_sample.set_properties(**{
    'width': '250px',
    'text-align': 'left',
    'vertical-align': 'top'
})
styled_sample.set_table_styles([
    {'selector': 'th', 'props': [('text-align', 'left')]}
])

display(styled_sample)


Przykładowe 20 wierszy z wynikami:


,Title,locationPath,Original_Location_Name,Predict_Loc
708417,"Apartament 60m2, Biuro Sprzedaży Dewelopera, 0%PCC","2,0,0,355,0,0,0",Śląskie > ? > ? > Katowice > ? > ? > ?,Katowice > ? > ?
660023,Mieszkanie 3 pokojowe,"16,0,0,366,3280,95200,0",Dolnośląskie > ? > ? > Wrocław > Wrocław-śródmieście > Śródmieście > ?,Wrocław > Śródmieście > ?
938398,Wysoki Standard | 3 Pokoje| 1Ha Park!,"14,0,0,337,3257,85909,368936",Małopolskie > ? > ? > Kraków > Kraków-krowodrza > Prądnik biały > Piaszczysta,Kraków > Prądnik biały > Piaszczysta
795676,Syndyk sprzeda lokal mieszkalny w Andrychowie Andrychów,"14,248,0,20719,0,0,416486",Małopolskie > Wadowicki > ? > Andrychów > ? > ? > Konstytucji 3 maja,Andrychów > ? > ?
304381,OGRÓD nad Motławą 322tys Netto pod INWESTYCJĘ 0pcc,"11,0,0,373,0,0,0",Pomorskie > ? > ? > Gdańsk > ? > ? > ?,Gdańsk > ? > ?
55946,Mieszkanie na Pogodnie,"5,0,0,396,0,0,292159",Zachodniopomorskie > ? > ? > Szczecin > ? > ? > Jana styki,Szczecin > ? > ?
6760,2-pokojowe mieszkanie na sprzedaż,"10,0,0,374,0,91395,0",Kujawsko-pomorskie > ? > ? > Bydgoszcz > ? > Okole > ?,Bydgoszcz > ? > ?
333209,"Mieszkanie do kapitalnego remontu - 44, 8 m2 Włocławek","10,0,0,346,0,0,0",Kujawsko-pomorskie > ? > ? > Włocławek > ? > ? > ?,Włocławek > ? > ?
319131,3-Pokojowe Bezczynszowe Mieszkanie Ogródek Garaż,"9,0,0,368,3269,98262,398604",Mazowieckie > ? > ? > Warszawa > Wawer > Zerzeń > Wiązana,Warszawa > Wawer > Bursztynowa
151293,"Mieszkanie, Piaseczno, 73 m²","9,326,0,51330,0,0,0",Mazowieckie > Piaseczyński > ? > Piaseczno > ? > ? > ?,Piaseczno > ? > ?
